<a href="https://colab.research.google.com/github/pallavi-vishnu/moma-dataset/blob/main/moma_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'museum-collection:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F836%2F1535%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240430%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240430T004412Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D83dbc6c98b27815c885bbe11c21b592531be8ac4d0408617b0ea5b665e5d853e5c9509a11c586e9c5acd48961bf6f39c9a3e5e6d5349d7e0785c1448473f76a742bf7ca41295daca19798efe233d3688966428fd66e9ae747fbf4ba95b0cfcfeb9b957179ab7bc956f4220267ff6a5ba7dfabe6c0354237e467a1ac4d80769d63531c450efab00d78822e291eb4589adfee3d4196621b6009552a153e5bb20e21dea3ffe4e35065038ac58bf9c5bd566cbdba7c314dcf02bbe4c1ab5ece1b901652ad40bdcf7cd589dcd70aefde3d82143a6c188d6063ead423746bbaf92c96c61f25719b8893942170725216af9360be363d0f49b6bf2d0a62d914d1818c2cc'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 6199715 bytes downloaded
Downloaded and uncompressed: museum-collection
Data source import complete.


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/museum-collection/artists.csv
/kaggle/input/museum-collection/artworks.csv


In [4]:
# Read the csv files
artist = pd.read_csv('/kaggle/input/museum-collection/artists.csv')
artwork = pd.read_csv('/kaggle/input/museum-collection/artworks.csv')

This Data contains data about artists and their artwork. We have to join the two datasets to get meaningful insight from the data.

In [5]:
# First lets check the shape of both the tables
print(artist.shape)
print(artwork.shape)

(15091, 6)
(130262, 21)


Looks like the number of rows in the artwork table is way more than that in the artist table. That's probably because some artists may have contributed to more than one artwork. Let's take a closer look at the information in both the tables.

In [6]:
artist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15091 entries, 0 to 15090
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Artist ID    15091 non-null  int64  
 1   Name         15091 non-null  object 
 2   Nationality  12603 non-null  object 
 3   Gender       12019 non-null  object 
 4   Birth Year   11237 non-null  float64
 5   Death Year   4579 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 707.5+ KB


In [7]:
artwork.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130262 entries, 0 to 130261
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Artwork ID          130262 non-null  int64  
 1   Title               130200 non-null  object 
 2   Artist ID           128802 non-null  object 
 3   Name                128802 non-null  object 
 4   Date                127950 non-null  object 
 5   Medium              118343 non-null  object 
 6   Dimensions          118799 non-null  object 
 7   Acquisition Date    124799 non-null  object 
 8   Credit              127192 non-null  object 
 9   Catalogue           130262 non-null  object 
 10  Department          130262 non-null  object 
 11  Classification      130262 non-null  object 
 12  Object Number       130262 non-null  object 
 13  Diameter (cm)       1399 non-null    float64
 14  Circumference (cm)  10 non-null      float64
 15  Height (cm)         111893 non-nul

In [8]:
art_table = pd.merge(artist, artwork, on = 'Name')
art_table.head()

,Artist ID_x,Name,Nationality,Gender,Birth Year,Death Year,Artwork ID,Title,Artist ID_y,Date,...,Classification,Object Number,Diameter (cm),Circumference (cm),Height (cm),Length (cm),Width (cm),Depth (cm),Weight (kg),Duration (s)
0,1,Robert Arneson,American,Male,1930.0,1992.0,33599,Study for Head Bath,1,1977,...,Drawing,67.1981,NaN,NaN,105.7,NaN,75.9,NaN,NaN,NaN
1,1,Robert Arneson,American,Male,1930.0,1992.0,64139,General Nuke,1,1986,...,Print,192.1997,NaN,NaN,80.5,NaN,59.2,NaN,NaN,NaN
2,2,Doroteo Arnaiz,Spanish,Male,1936.0,NaN,61629,BAS-RELIEF,2,NaN,...,Print,87.1965,NaN,NaN,54.7,NaN,39.7,NaN,NaN,NaN
3,3,Bill Arnold,American,Male,1941.0,NaN,45972,Honey under Sink,3,1971,...,Photograph,87.1972,NaN,NaN,38.1,NaN,58.2,NaN,NaN,NaN
4,3,Bill Arnold,American,Male,1941.0,NaN,45997,Honey under Chair,3,1971,...,Photograph,88.1972,NaN,NaN,38.1,NaN,57.8,NaN,NaN,NaN


In [9]:
# make a copy of the original dataset

moma_data = art_table.copy()

In [16]:
# Exctract the necessary columns

moma_data_filtered = moma_data[['Artist ID_x', 'Name', 'Nationality', 'Gender', 'Artwork ID', 'Title', 'Date', 'Acquisition Date', 'Credit', 'Department', 'Classification', ]]

In [17]:
moma_data_filtered

,Artist ID_x,Name,Nationality,Gender,Artwork ID,Title,Date,Acquisition Date,Credit,Department,Classification
0,1,Robert Arneson,American,Male,33599,Study for Head Bath,1977,1981-04-28,Gift of the Friends of Contemporary Drawing,Drawings,Drawing
1,1,Robert Arneson,American,Male,64139,General Nuke,1986,1997-05-28,Gift of Landfall Press,Prints & Illustrated Books,Print
2,2,Doroteo Arnaiz,Spanish,Male,61629,BAS-RELIEF,NaN,1965-03-09,Gift of the artist,Prints & Illustrated Books,Print
3,3,Bill Arnold,American,Male,45972,Honey under Sink,1971,1972-03-07,Purchase,Photography,Photograph
4,3,Bill Arnold,American,Male,45997,Honey under Chair,1971,1972-03-07,Purchase,Photography,Photograph
...,...,...,...,...,...,...,...,...,...,...,...
138023,67379,N. Dash,American,Female,215840,Hot Eye,2016,2016-11-14,Acquired through the generosity of Mary M. and...,Prints & Illustrated Books,Print
138024,67420,Walter Burley Griffin,NaN,NaN,216180,"Planting Plan for Darwin Martin House, Buffalo...",1905,NaN,NaN,Architecture & Design,Frank Lloyd Wright Archive
138025,67437,Sebastian Black,American,Male,216351,Tails,2016,2016-11-14,Anonymous gift,Prints & Illustrated Books,Illustrated Book
138026,67694,Glenn Williams,NaN,Male,217974,Text used for Martin,NaN,2008-10-08,The Gilbert and Lila Silverman Fluxus Collecti...,Fluxus Collection,Media
